In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. 관심도가 낮은지 중간인지 높은지 분류하는 대회 
2. read_json 으로 가져오기
3. 글을 길게 적으면 사람들이 관심을 갖지 않는다고 말해주고, 사진을 올려야 관심도가 높아진다고 말해줄 수 있음 (왜 사진 너무 많이 올리면 안좋은지 생각해보기)  


# Table of Contents 
* [Baseline](#Chapter1)
    * [데이터 가져오기](#Section_1_1)
    * [photo 숫자로 바꾸기,Description길이로 바꾸기,날짜 바꾸기 ](#Section_1_2)
    * [ID drop하기](#Section_1_3)
    * [Features column 추가 컬럼 만들기](#Section_1_4)
    * [dummy 변수 처리하기](#Section_1_5)
    * [결측치 처리하기](#Section_1_6)
* [EDA](#Chapter2)
    * [Correlation 확인하기](#Section_2_0)
    * [Interest Level 분포 확인하기](#Section_2_1)
    * [날짜 변수 분포 확인하기](#Section_2_2)
* [Training](#Chapter3)
    * [Catboost](#Section_3_1)
    * [Random Forest](#Section_3_2)
----------------------
Questions

-----------------------
* [question1](#Q1)
* [question2](#Q2)
* [question3](#Q3)
* [question4](#Q4)
* [question4](#Q5)


------------------
Score Board (the higher score the better)

---------------
1. Baseline: 2.69942
2. Description, Photo 개수: 2.87189
3. Catboost: 1.87623

# 1.Baseline <a class='anchor' id='Chapter1'></a>

## 1-1.데이터 가져오기 <a class='anchor' id='Section_1_1'></a>

In [ ]:
train = pd.read_json('/kaggle/input/two-sigma-connect-rental-listing-inquiries/train.json.zip')
test = pd.read_json('/kaggle/input/two-sigma-connect-rental-listing-inquiries/test.json.zip')

In [ ]:
all_data = pd.concat([train,test])

In [ ]:
all_data.head(15)

In [ ]:
tree모델은 column 많아지면 차원의 저주 발생해서  - tree모델 안좋음 (boosting도 tree)
tree(bagging-randomforest, boosting 두가지 종류)

tree모델은 column 많아지면 차원의 저주 발생해서  - tree모델 안좋음 (boosting도 tree)
tree(bagging-randomforest, boosting 두가지 종류)

## 1-2.photo 숫자로 바꾸기,Description길이로 바꾸기,날짜 바꾸기 <a class='anchor' id='Section_1_2'></a>

In [ ]:
all_data['photos'] = all_data['photos'].apply(lambda x: len(x))

In [ ]:
all_data['description'] = all_data['description'].apply(lambda x: len(str(x)))

In [ ]:
all_data.head(2)

In [ ]:
all_data['created'] = pd.to_datetime(all_data['created'])
all_data['year'] = all_data['created'].dt.year
all_data['month'] = all_data['created'].dt.month
all_data['day'] = all_data['created'].dt.day
all_data['hour'] = all_data['created'].dt.hour

In [ ]:
all_data.head(2)

## 1-3.ID drop하기 <a class='anchor' id='Section_1_3'></a>

In [ ]:
all_data = all_data.drop(columns=['building_id','manager_id'])

In [ ]:
all_data.info()

## 1-4.Features column 추가 컬럼 만들기 <a class='anchor' id='Section_1_4'></a>

In [ ]:
all_data['features'] = all_data['features'].apply(lambda x: list(x))

In [ ]:
type(all_data['features'][0])
all_data['features_num'] = all_data['features'].apply(lambda x: len(x))
all_data

## 1-5.dummy 변수 처리하기 <a class='anchor' id='Section_1_5'></a>
질문1. 만약에 Catboost 쓰려면 더미 처리해야하는데 왜 안되지 ? ㅠㅠ <a class='anchor' id='Q1'></a>

In [ ]:
#all_data = pd.get_dummies(all_data)

In [ ]:
#all_data = pd.get_dummies(all_data)

## 1-6.결측치 처리하기 <a class='anchor' id='Section_1_6'></a>

In [ ]:
all_data.isnull().sum()

# 2.EDA <a class='anchor' id='Chapter2'></a>

## 2-0.Correlation 확인하기 <a class='anchor' id='Section_2_0'></a>

In [ ]:
group = all_data.groupby(all_data['interest_level']).mean()
group

## 2-1.Interest Level 분포 확인하기 <a class='anchor' id='Section_2_1'></a>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(all_data['interest_level'])

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(all_data['photos'],hue=all_data['interest_level'])

### 사진의 수는 4,5,6개가 가장 많고, 사진 갯수가 12개부터는 부정적인 영향을 줌, 0일 때는 부정적인 결과 나온다는 것도 학습할 수 있음

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(all_data['description'],hue=all_data['interest_level'])

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(all_data['features_num'],hue=all_data['interest_level'])

In [ ]:
sns.scatterplot(data=all_data, x="photos", y="features_num", hue="interest_level")

In [ ]:
plt.figure(figsize=(12,20))
sns.catplot(x='interest_level',y='price', data=all_data)

### 가격이 너무 높을 경우 interst level이 낮음

## 2-2.날짜 변수 분포 확인하기<a class='anchor' id='Section_2_2'></a>

질문: for 문으로 2x2 그래프 그리기 <a class='anchor' id='Q2'></a>

In [ ]:
plt.figure(figsize=(20,20))
for i, item in enumerate(['year','month','day','hour']):
    plt.subplot(220+(i+1))
    sns.countplot(all_data[item], hue=all_data['interest_level'])

In [ ]:
_ , b = plt.subplots(4, 1, figsize=(10,15))

for i, item in enumerate(['year','month','day','hour']):
    sns.countplot(all_data[item], hue=all_data['interest_level'], ax=b[i])
            

# sns.countplot(all_data['year'],hue=all_data['interest_level'],ax=b[0][0])
# sns.countplot(all_data['month'],hue=all_data['interest_level'],ax=b[0][1])
# sns.countplot(all_data['day'],hue=all_data['interest_level'],ax=b[1][0])
# sns.countplot(all_data['hour'],hue=all_data['interest_level'],ax=b[1][1])

### *hour이 정말 중요한 변수라는 걸 알 수 있음 

# 3. Training <a class='anchor' id='Chapter3'></a>
 
질문: Catboost랑 randomforest 장단점 -> Catboost가 더 튜닝 다양하게 가능하고, 더 많은 컬럼에 유리하면 언제나 Catboost가 Better? <a class='anchor' id='Q3'></a>
-> 대부분의 경우 boosting모델의 option 설정 잘하면 잘 나옴. 데이터에 대한 파악이 어려울 땐 Random Forest(옵션에 대한 설정이 어려울테니까) 
-> Catboost는 옵션 설정이 boosting 모델 중에선 구체적으로 안해줘도 점수 잘 나옴

In [ ]:
all_data

In [ ]:
all_data2 = all_data.drop(columns=['interest_level','created','features','street_address','display_address','features_num','photos'])

In [ ]:
all_data = all_data.fillna(-1)

In [ ]:
train2 = all_data2[:len(train)]
target2 = all_data[:len(train)]['interest_level']
display(train2, target2, target2.isnull().sum())

## 3-1.Catboost <a class='anchor' id='Section_3_1'></a>

In [ ]:
all_data['interest_level'].describe()

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_valid, Y_train, Y_valid = train_test_split(train2, all_data[:len(train)]['interest_level'], test_size=0.2, random_state=123)


In [ ]:
from catboost import CatBoostClassifier 
cbc = CatBoostClassifier(task_type='GPU')
cbc.fit(X_train, Y_train, eval_set=(X_valid, Y_valid))
result=cbc.predict_proba(X_valid)
result


In [ ]:
result1 = cbc.predict_proba(all_data2[len(train):])

질문: 왜 안되지? 두 개 모델 다 <a class='anchor' id='Q4'></a>

## 3-2.Random Forest<a class='anchor' id='Section_3_2'></a>

질문: 여기에 category '-1'은 빠졌는지 어뜨케 앎? predict 중에서.. <a class='anchor' id='Q5'></a>

-> test 셋에는 결측치 없음 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,n_jobs=-1, random_state=12)
rf.fit(train2, target2)
result2 = rf.predict_proba(all_data2[len(train):])

In [ ]:
result

In [ ]:
sub = pd.read_csv('/kaggle/input/two-sigma-connect-rental-listing-inquiries/sample_submission.csv.zip')

In [ ]:
sub

In [ ]:
sub['high'] = result1[:,0]
sub['medium'] = result1[:,1]
sub['low'] = result1[:,2]

In [ ]:
sub

In [ ]:
sub.to_csv('submission.csv', index=False)